**Градиентный бустинг (gradient boosting)** — это метод машинного обучения, который позволяет последовательно улучшать качество предсказаний, комбинируя слабые модели (например, решающие деревья) в сильную модель.

Основная идея заключается в том, чтобы обучить новую модель, которая будет исправлять "ошибки" предыдущей модели, постепенно улучшая качество предсказаний на каждой итерации.

Таким образом, градиентный бустинг является итеративным методом, который обучает модели последовательно. В результате ансамбль моделей становится все более точным и дает лучшие предсказания, чем отдельно взятая слабая модель.


**Первый базовый алгоритм**


In [1]:
import numpy as np
import pandas as pd


class GradientBoostingRegressor:
    """Gradient boosting regressor."""

    def fit(self, X, y):
        """Fit the model to the data.

        Args:
            X: array-like of shape (n_samples - x.shape[0], n_features - x.shape[1])
            y: array-like of shape (n_samples,)

        Returns:
            GradientBoostingRegressor: The fitted model.
        """
        #среднее значение по выборке. Сохраните среднее значение в дополнительный атрибут класса self.base_pred_
        self.base_pred_ = np.mean(y)
        return self

    def predict(self, X):
        """Predict the target of new data.

        Args:
            X: array-like of shape (n_samples, n_features)

        Returns:
            y: array-like of shape (n_samples,)
            The predict values.

        """
        # делал прогноз для всех значений средним значением по обучающей выборке.
        if hasattr(X, "__len__"):
            n = len(X)
        else:
            n = X.shape[0]

        predictions = np.full(n,self.base_pred_)

        return predictions

In [2]:
X = np.random.randn(5, 3)
y = np.array([10, 12, 13, 14, 15])

model = GradientBoostingRegressor()
model.fit(X, y)

print(model.base_pred_)  # Должно быть 12.8
print(model.predict(X))  # Массив из 12.8 длины 5

12.8
[12.8 12.8 12.8 12.8 12.8]


**Мы разобрали, что каждое новое дерево в ансамбле обучается на суммарной ошибке предыдущих деревьев.**

Поэтому любую константу в векторе градиента можно сократить, потому что она компенсируется другим значением шага кратным этой константе.

In [ ]:
import numpy as np
from typing import Tuple


def mse(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, np.ndarray]:
    """Mean squared error loss function and gradient."""
    # YOUR CODE HERE
    # loss - значение функции потерь (усредненное по всем объектам)
    # grad - вектор псевдо-остатков для каждого объекта
    loss = np.mean(y_pred - y_true) ** 2
    grad = y_pred - y_true
    return loss, grad

def mae(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, np.ndarray]:
    """Mean absolute error loss function and gradient."""
    # YOUR CODE HERE
    loss = np.mean(np.abs(y_pred - y_true))
    grad = np.sign(y_pred - y_true)
    return loss, grad

**Доработать GradBoost**

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

class GradientBoostingRegressor:
    def __init__(
        self,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        loss="mse",
        verbose=False,
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.loss = loss
        self.verbose = verbose
        self.base_pred_ = None
        self.trees_ = []

    def _mse(self, y_true, y_pred):
        # loss (усредненная), градиент (остатки)
        loss = np.mean((y_true - y_pred) ** 2)
        grad = y_pred - y_true
        return loss, grad

    def fit(self, X, y):
        self.base_pred_ = np.mean(y)
        y_pred = np.full_like(y, self.base_pred_, dtype=float)
        self.trees_ = []

        # Определяем функцию потерь и градиент
        if callable(self.loss):
            loss_func = self.loss
        elif self.loss == "mse":
            loss_func = self._mse
        else:
            raise ValueError(f"Unknown loss: {self.loss}")
        for i in range(self.n_estimators):
            # 1. Считаем градиент (антиградиент)
            loss, grad = loss_func(y, y_pred)
            anti_grad = -grad
            # 2. Обучаем дерево на антиградиентах
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                         min_samples_split=self.min_samples_split)
            tree.fit(X, anti_grad)
            # 3. Вычисляем предсказания дерева, делаем шаг градиентного спуска
            tree_pred = tree.predict(X)
            y_pred += self.learning_rate * tree_pred
            # 4. Сохраняем дерево
            self.trees_.append(tree)
            # 5. Если надо, выводим loss
            if self.verbose:
                current_loss, _ = loss_func(y, y_pred)
                print(f"Step {i+1}/{self.n_estimators}, Loss: {current_loss:.5f}")
        return self

    def predict(self, X):
        # 1. Предсказание средним
        n = X.shape[0]
        y_pred = np.full(n, self.base_pred_, dtype=float)
        for tree in self.trees_:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

**Cтохастический градиентный спуск (SGD — stochastic gradient descent). Стохастический — т.е. случайный.**

Использование SGD помогает внести случайность и разнообразие в построение каждого дерева, что может снизить переобучение и улучшить обобщающую способность модели.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

class GradientBoostingRegressor:
    def __init__(
        self,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        loss="mse",
        verbose=False,
        subsample_size=0.5,
        replace=False,
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.loss = loss
        self.verbose = verbose
        self.subsample_size = subsample_size
        self.replace = replace
        self.base_pred_ = None
        self.trees_ = []

    def _mse(self, y_true, y_pred):
        loss = np.mean((y_true - y_pred) ** 2)
        grad = y_pred - y_true
        return loss, grad

    def _subsample(self, X, y):
        n_samples = X.shape[0]
        sample_size = int(self.subsample_size * n_samples)
        idx = np.random.choice(n_samples, size=sample_size, replace=self.replace)
        return X[idx], y[idx]

    def fit(self, X, y):
        self.base_pred_ = np.mean(y)
        y_pred = np.full_like(y, self.base_pred_, dtype=float)
        self.trees_ = []

        if callable(self.loss):
            loss_func = self.loss
        elif self.loss == "mse":
            loss_func = self._mse
        else:
            raise ValueError(f"Unknown loss: {self.loss}")

        for i in range(self.n_estimators):
            loss, grad = loss_func(y, y_pred)
            anti_grad = -grad

            sub_X, sub_anti_grad = self._subsample(X, anti_grad)

            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(sub_X, sub_anti_grad)

            tree_pred = tree.predict(X)
            y_pred += self.learning_rate * tree_pred
            self.trees_.append(tree)

            if self.verbose:
                current_loss, _ = loss_func(y, y_pred)
                print(f"Step {i+1}/{self.n_estimators}, Loss: {current_loss:.5f}")
        return self

    def predict(self, X):
        n = X.shape[0]
        y_pred = np.full(n, self.base_pred_, dtype=float)
        for tree in self.trees_:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred